[View in Colaboratory](https://colab.research.google.com/github/iamrajee/Data-Analysis/blob/master/cv_ner_on_bow.ipynb)

In [0]:
!ls

1.txt  2.txt  3.txt  4.txt  5.txt  datalab  nltk_data


In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
!pip install gensim
!pip install spacy
!pip install pyLDAvis


In [0]:
#@title Default title text
import re
import numpy as np
import pandas as pd


#nltk
import nltk
nltk.download('all')


#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy for lemmatization
import spacy

#plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline


#optional
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /content/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /content/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /content/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /content/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /content/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to

In [0]:
raw_corpus = []
for i in range(1,5):
    filename = str(i) + '.txt'
    f = open(filename,'r')#file opened
    temp = f.read()
    #raw_corpus[i] = temp
    raw_corpus.append(temp)
#print(raw_corpus)
#print(np.shape(raw_corpus))

In [0]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /content/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /content/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /content/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /content/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /content/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to

True

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize

#EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."
raw_tokens = []
for i, text in enumerate(raw_corpus):
  raw_tokens.append(word_tokenize(text))
import numpy as np
print(np.shape(raw_tokens))
print(raw_tokens)


(4,)
[['LETTIERI', 'Maurizio', 'Date', 'of', 'birth', '17', 'February', '1979', 'Nationality', 'Italian', 'Contact', 'details', 'Email', ':', 'maurizio.lettieri', '@', 'alumni.insead.edu', 'Tel', ':', '+33687279684', '(', 'until', 'July07', ')', 'Mobile', ':', '+393488889498', 'Permanent', 'address', 'Viale', 'Gallerani', '6', '21047', 'Saronno', 'VA', 'Italy', 'SUMMARY', 'Seeking', 'a', 'position', 'in', 'a', 'dynamic', 'and', 'team', 'oriented', 'environment', 'whereby', 'I', 'will', 'be', 'assessed', 'against', 'challenging', 'objectives', 'PROFESSIONAL', 'EXPERIENCE', '2004', '-', '2006', 'McKinsey', 'and', 'Company', 'Management', 'Consulting', 'Business', 'Analyst', 'Italy', 'Strategic', 'Marketing', '•', 'Defined', 'long-term', 'marketing', 'strategy', 'for', 'a', 'leading', 'telecommunication', 'company', '•', 'Developed', 'a', 'business', 'plan', 'for', 'three', 'main', 'segments', 'and', 'assessed', 'opportunities', 'for', 'incumbent', 'and', 'new', 'entrants', '(', 'virtual'

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

stop_words.extend(['from', 'use', 'also', 'e.g.','m','•'])#,'j.', 'm.','self','make', 'likely', 'a.', '..', 'l.', 'l', 'of'])
# stop_words.extend(['firm','ceo','work','may','organizational','organization'])

stop_tokens = []
for i, doctoken in enumerate(raw_tokens):
  stop_tokens.append([w for w in doctoken if not w in stop_words])
print(list(stop_tokens))

[['LETTIERI', 'Maurizio', 'Date', 'birth', '17', 'February', '1979', 'Nationality', 'Italian', 'Contact', 'details', 'Email', ':', 'maurizio.lettieri', '@', 'alumni.insead.edu', 'Tel', ':', '+33687279684', '(', 'July07', ')', 'Mobile', ':', '+393488889498', 'Permanent', 'address', 'Viale', 'Gallerani', '6', '21047', 'Saronno', 'VA', 'Italy', 'SUMMARY', 'Seeking', 'position', 'dynamic', 'team', 'oriented', 'environment', 'whereby', 'I', 'assessed', 'challenging', 'objectives', 'PROFESSIONAL', 'EXPERIENCE', '2004', '-', '2006', 'McKinsey', 'Company', 'Management', 'Consulting', 'Business', 'Analyst', 'Italy', 'Strategic', 'Marketing', 'Defined', 'long-term', 'marketing', 'strategy', 'leading', 'telecommunication', 'company', 'Developed', 'business', 'plan', 'three', 'main', 'segments', 'assessed', 'opportunities', 'incumbent', 'new', 'entrants', '(', 'virtual', 'operator', ',', 'segment', 'focus', 'approach', ')', 'Sales', 'Increased', 'acquisition', '(', '150,000', 'new', 'clients', '€2

In [0]:
!python3 -m spacy download en

    100% |████████████████████████████████| 37.4MB 96.1MB/s 

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stem_tokens = []
for i, doctoken in enumerate(stop_tokens):
  if i == 0:
    stem_tokens.append([ps.stem(w) for w in doctoken])
  else:
    break
print(stem_tokens)

nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(text2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in text2:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
  
#Attention 
lemma_corpus2 = lemmatization(stop_tokens, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV'])
lemma_corpus = stop_tokens    #Attention
print(np.shape(lemma_corpus))
print(lemma_corpus)

[['lettieri', 'maurizio', 'date', 'birth', '17', 'februari', '1979', 'nation', 'italian', 'contact', 'detail', 'email', ':', 'maurizio.lettieri', '@', 'alumni.insead.edu', 'tel', ':', '+33687279684', '(', 'july07', ')', 'mobil', ':', '+393488889498', 'perman', 'address', 'vial', 'gallerani', '6', '21047', 'saronno', 'VA', 'itali', 'summari', 'seek', 'posit', 'dynam', 'team', 'orient', 'environ', 'wherebi', 'I', 'assess', 'challeng', 'object', 'profession', 'experi', '2004', '-', '2006', 'mckinsey', 'compani', 'manag', 'consult', 'busi', 'analyst', 'itali', 'strateg', 'market', 'defin', 'long-term', 'market', 'strategi', 'lead', 'telecommun', 'compani', 'develop', 'busi', 'plan', 'three', 'main', 'segment', 'assess', 'opportun', 'incumb', 'new', 'entrant', '(', 'virtual', 'oper', ',', 'segment', 'focu', 'approach', ')', 'sale', 'increas', 'acquisit', '(', '150,000', 'new', 'client', '€200', 'million', 'addit', 'valu', ')', 'lead', 'european', 'bank', 'group', '-', 'creat', 'new', 'team'

In [0]:
bigram = gensim.models.Phrases(lemma_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram = gensim.models.Phrases(bigram_mod[lemma_corpus], threshold=100) # higher threshold fewer phrases.
trigram_mod = gensim.models.phrases.Phraser(trigram)
trigram = gensim.models.Phrases(trigram_mod[lemma_corpus], threshold=100) # higher threshold fewer phrases.
corpus_quad = gensim.models.phrases.Phraser(trigram)


def make_bigrams(text2):
    return [bigram_mod[doc] for doc in text2]
def make_trigrams(text2):
    return [trigram_mod[bigram_mod[doc]] for doc in text2] 
def make_quadgrams(text2):
    return [trigram_mod[bigram_mod[doc]] for doc in text2] 
  
  
corpus_bi = make_bigrams(lemma_corpus)
corpus_tri = make_trigrams(lemma_corpus)

#raw_tokens
corpus_bi_raw = make_bigrams(raw_tokens)
corpus_tri_raw = make_trigrams(raw_tokens)

#corpus_quad = make_quadgrams(lemma_corpus)#there is no change in len so use corpus_tri

print(len(corpus_tri[0]))
print(corpus_tri)

376
[['LETTIERI', 'Maurizio', 'Date', 'birth', '17', 'February', '1979', 'Nationality', 'Italian', 'Contact', 'details', 'Email', ':', 'maurizio.lettieri', '@', 'alumni.insead.edu', 'Tel', ':', '+33687279684', '(', 'July07', ')', 'Mobile', ':', '+393488889498', 'Permanent', 'address', 'Viale', 'Gallerani', '6', '21047', 'Saronno', 'VA', 'Italy', 'SUMMARY', 'Seeking', 'position', 'dynamic', 'team', 'oriented', 'environment', 'whereby', 'I', 'assessed', 'challenging', 'objectives', 'PROFESSIONAL', 'EXPERIENCE', '2004', '-', '2006', 'McKinsey', 'Company', 'Management', 'Consulting', 'Business', 'Analyst', 'Italy', 'Strategic', 'Marketing', 'Defined', 'long-term', 'marketing', 'strategy', 'leading', 'telecommunication', 'company', 'Developed', 'business', 'plan', 'three', 'main', 'segments', 'assessed', 'opportunities', 'incumbent', 'new', 'entrants', '(', 'virtual', 'operator', ',', 'segment', 'focus', 'approach', ')', 'Sales', 'Increased', 'acquisition', '(', '150,000', 'new', 'clients',

In [0]:
print(np.shape(raw_tokens))
print(raw_tokens)
print(np.shape(corpus_tri))
print(corpus_tri)
# print(np.shape())
# print(np.shape())

(4,)
[['LETTIERI', 'Maurizio', 'Date', 'of', 'birth', '17', 'February', '1979', 'Nationality', 'Italian', 'Contact', 'details', 'Email', ':', 'maurizio.lettieri', '@', 'alumni.insead.edu', 'Tel', ':', '+33687279684', '(', 'until', 'July07', ')', 'Mobile', ':', '+393488889498', 'Permanent', 'address', 'Viale', 'Gallerani', '6', '21047', 'Saronno', 'VA', 'Italy', 'SUMMARY', 'Seeking', 'a', 'position', 'in', 'a', 'dynamic', 'and', 'team', 'oriented', 'environment', 'whereby', 'I', 'will', 'be', 'assessed', 'against', 'challenging', 'objectives', 'PROFESSIONAL', 'EXPERIENCE', '2004', '-', '2006', 'McKinsey', 'and', 'Company', 'Management', 'Consulting', 'Business', 'Analyst', 'Italy', 'Strategic', 'Marketing', '•', 'Defined', 'long-term', 'marketing', 'strategy', 'for', 'a', 'leading', 'telecommunication', 'company', '•', 'Developed', 'a', 'business', 'plan', 'for', 'three', 'main', 'segments', 'and', 'assessed', 'opportunities', 'for', 'incumbent', 'and', 'new', 'entrants', '(', 'virtual'

In [0]:
np.shape(corpus_tri_raw)

(4,)

In [0]:
pos_corpus = []
namedEnt = []
for i, doctoken in enumerate(corpus_tri_raw):
  if i == 0:
    pos_corpus.append(nltk.pos_tag(doctoken))
    namedEnt.append(nltk.ne_chunk(pos_corpus[i], binary=False))
  else:
    break

# extract all named entities
named_entities = []
entity_typelist = []
for i, ne_chunked_sents in enumerate(namedEnt):
  named_entitiestemp = []
  if i == 0:
    for tagged_tree in ne_chunked_sents:
      if hasattr(tagged_tree, 'label'):
        entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
        entity_type = tagged_tree.label() # get NE category
        entity_typelist.append(entity_type)
        named_entitiestemp.append((entity_name, entity_type))
    named_entities.append(named_entitiestemp)
  else:
    break
entity_typelist = np.unique(entity_typelist) 
# print(entity_typelist)
print(named_entities)



GPElist = []
ORGlist = []
PERSONlist = []
for i, doctoken in enumerate(named_entities):
  if i == 0:
    for w,wtype in doctoken:
      #print(wtype)
      if wtype == 'GPE':
        GPElist.append(w)
      elif wtype == 'ORGANIZATION':
        ORGlist.append(w)
      elif wtype == 'PERSON':
        PERSONlist.append(w)
    
  else:
    break

    
GPElist = np.unique(GPElist)
ORGlist = np.unique(ORGlist)
PERSONlist = np.unique(PERSONlist)
print('\n\nGPElist length:' + str(len(GPElist)))
print('ORGlist length:' + str(len(ORGlist)))
print('PERSONlist length:' + str(len(PERSONlist)))

print('\n\n\nGPE : ' + str(GPElist))
print('ORGANIZATION :' + str(ORGlist))
print('PERSON :' + str(PERSONlist))

[[('LETTIERI', 'ORGANIZATION'), ('Maurizio Date', 'PERSON'), ('Italian Contact', 'PERSON'), ('Email', 'PERSON'), ('Tel', 'GPE'), ('July07', 'GPE'), ('Viale Gallerani', 'PERSON'), ('Saronno VA Italy', 'PERSON'), ('PROFESSIONAL', 'ORGANIZATION'), ('McKinsey', 'ORGANIZATION'), ('Company Management Consulting Business Analyst Italy', 'ORGANIZATION'), ('Increased', 'ORGANIZATION'), ('European Banking Group', 'ORGANIZATION'), ('Tools', 'GPE'), ('Information Systems', 'ORGANIZATION'), ('European Banking Group', 'ORGANIZATION'), ('European Banking Group', 'ORGANIZATION'), ('Cargo Division', 'ORGANIZATION'), ('LANGUAGES', 'ORGANIZATION'), ('INSEAD', 'ORGANIZATION'), ('Milano Laurea', 'PERSON'), ('Industrial', 'GPE'), ('France', 'GPE'), ('Italy Italian', 'PERSON'), ('English', 'GPE'), ('Business', 'ORGANIZATION'), ('Spanish', 'GPE'), ('Global Management Challenge Honk Kong', 'PERSON'), ('Portugal', 'GPE'), ('Italian', 'GPE'), ('Macao-Hong Kong', 'ORGANIZATION'), ('Madeira', 'ORGANIZATION'), ('It

In [0]:
from nltk.corpus import wordnet

synonyms = []
antonyms = []

for syn in wordnet.synsets("good"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(set(synonyms))
print(set(antonyms))

{'skillful', 'salutary', 'beneficial', 'commodity', 'dependable', 'dear', 'right', 'trade_good', 'secure', 'in_force', 'goodness', 'honest', 'unspoilt', 'well', 'upright', 'in_effect', 'serious', 'honorable', 'proficient', 'undecomposed', 'estimable', 'effective', 'respectable', 'practiced', 'unspoiled', 'adept', 'good', 'ripe', 'safe', 'near', 'thoroughly', 'full', 'soundly', 'skilful', 'expert', 'just', 'sound'}
{'badness', 'evil', 'bad', 'ill', 'evilness'}


In [0]:
w1 = wordnet.synset('ship.n.01')
w2 = wordnet.synset('car.n.01')
print(w1.wup_similarity(w2))

0.6956521739130435
